# Atomic structure analysis with radial (azimuthal) average & variance profiles
### Jinseok Ryu (jinseok.ryu@diamond.ac.uk)
Only compatible with the ePSIC data processig workflow  
Recommended to run this notebook using Python 3.10 EPSIC kernel on the jupyterhub of Diamond Light Source  
Otherwise, it is highly probable that it will not work properly  
[Required Python packages]  
scipy, numpy, matplotlib, py4DSTEM, hyperspy, drca (optional for NMF, https://github.com/jinseuk56/drca)

In [ ]:
import sys
sys.path.append('/~/script')

from radial_profile_analysis import *

In [ ]:
# load data
base_dir = '/dls/e02/data/2025/mgXXXXX-X'
subfolders = [''] # subfolder names you want to load and compare, e.g., ['sub1', 'sub2']
final_dir = None # (optional) folder name where the data is stored in

profile_length = 360 # limit the profile size
num_load = 2 # limit the number of data for every subfolder (select files randomly)

include_key = [] # keyword (datetime) for screening (to only include the specified data)
exclude_key = [] # keyword (datetime) for screening (to exclude poor quality data)

specific_scan_shape = [] # (optional) if you want to load only the data with a specific scan shape

# The path of EDX data should be like this: ~/EDX/*.rpl
run_analysis = radial_profile_analysis(base_dir, subfolders, 
                                       profile_length, num_load, final_dir,
                                       include_key, exclude_key, specific_scan_shape,
                                       simult_edx=True, roll_axis=True, verbose=False)

In [ ]:
plt.close("all")
# Transformation quality check (center beam alignment)
# If there are any data of poor quality, you can exclude them in the cell above (using 'exclude_key')
# crop=[top, bottom, left, right] -> img[top:bottom, left:right] (optional)
run_analysis.center_beam_alignment_check(crop=[0, -1, 0, -1], 
                                         visual_title=True)

In [ ]:
plt.close("all")
# Intensity integration image (BF + DF)
run_analysis.intensity_integration_image(visual_title=True)

In [ ]:
# To simulate diffraction patterns (XRD) from prismatic structure file(s) (optional)
str_path = [] # structure paths to compare, e.g., ['path1', 'path2']

# Specify the scattering vector range -> also used in NMF decomposition and plotting
from_unit = 0.2 # unit: 1/angstrom, it must be equal to or greater than zero
to_unit = 0.6 # unit: 1/angstrom, it must be smaller than the maximum scattering vector
run_analysis.basic_setup(str_path, from_unit, to_unit, 
                         broadening=0.01, fill_width=0.005) # broadening -> used to simulate diffraction patterns

In [ ]:
plt.close("all")
# Sum of radial variance and average profiles
# profile_type: "mean" or "variance"
# str_name=["structure_name_1", "structure_name_2"]
# The structure names are stored in run_analysis.int_sf.keys()
run_analysis.sum_radial_profile(str_name=[], 
                                profile_type="variance",
                               visual_legend=False)

# NMF decomposition

In [ ]:
# Optional process
# NMF - to optimize the number of loading vectors
# rescale_SI=True -> divide each 3D data by its maximum value
# max_normalize=True -> divide every profile by its maximum value
# rescale_0_to_1=True -> rescale every profile from 0 to 1
# Please refer to Scikit-learn, 'nmf' or 'https://github.com/jinseuk56/drca'
# profile_type: "mean" or "variance"
# verbose=True -> it will show the loading vectors and their corresponding coefficient maps
# coeff_map_type: "relative" or "absolute"
# coeff_map_type="absolute" -> the colormap range of all the coefficient maps will be determined from the maximum coefficient to the minimum coefficient

error_list = []
comp_list = []
num_comp_list = np.arange(2, 20, 2)

run_analysis.NMF_decompose(num_comp_list[0], profile_type="variance", 
                           rescale_SI=True, max_normalize=False, rescale_0to1=False, 
                           verbose=False, coeff_map_type="relative")
error_list.append(run_analysis.run_SI.DR.reconstruction_err_)
comp_list.append(run_analysis.run_SI.DR.components_)

for num_comp in num_comp_list[1:]:
    run_analysis.run_SI.ini_DR(method="nmf", num_comp=num_comp, result_visual=False, intensity_range="relative")
    error_list.append(run_analysis.run_SI.DR.reconstruction_err_)
    comp_list.append(run_analysis.run_SI.DR.components_)

# plot the errors between the original dataset and the reconstructed dataset
# according to the number of loading vectors
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.plot(num_comp_list, error_list, 'k-')
ax.plot(num_comp_list, error_list, 'r*')
ax.set_xlabel("Number of loading vectors")
ax.set_ylabel("Error")
fig.tight_layout()
plt.show()

# plot the loading vectors for each case
# for i, n_comp in enumerate(num_comp_list):
#     fig, ax = plt.subplots(1, 1, figsize=(6, 4))
#     for j, comp in enumerate(comp_list[i]):
#         ax.plot(comp+j*1.0)
#     fig.tight_layout()
#     plt.show()

In [ ]:
plt.close("all")
# NMF - to optimize the number of loading vectors
# rescale_SI=True -> divide each 3D data by its maximum value
# max_normalize=True -> divide every profile by its maximum value
# rescale_0_to_1=True -> rescale every profile from 0 to 1
# scale_crop=True -> the profiles will be cropped in a scattering vector range, otherwise in an index range
# Please refer to Scikit-learn, 'nmf' or 'https://github.com/jinseuk56/drca'
# profile_type: "mean" or "variance"
# verbose=True -> it will show the loading vectors and their corresponding coefficient maps
# coeff_map_type: "relative" or "absolute"
# 'relative' -> color scale for each 3D data, 'absolute'-> color scale for all 3D data

num_comp = 8
run_analysis.NMF_decompose(num_comp, profile_type="variance", scale_crop=True, rescale_SI=False,
                           max_normalize=False, rescale_0to1=True, 
                           verbose=True, coeff_map_type="relative")

In [ ]:
plt.close("all")
# NMF - loading vectors and their coefficient maps
# If the number of loading vectors exceeds the number of the preset colormaps (currently five),
# it will show the coefficient maps for only 5 loading vectors
# lv_show [loading vector number list] -> you can choose which coefficient map to show
# The colormap list can be accessed run_analysis.cm_rep -> new colormaps can be added
run_analysis.NMF_result(lv_show=None, visual_title=True, title_font_size=5)

In [ ]:
plt.close("all")
# NMF - show the pixels with high coefficients for each loading vector and the averaged profiles for those pixels
# str_name=["structure_name_1", "structure_name_2"]
# percentile_threshold -> if 90, only the pixels with the 10% highest coefficients remain
by_nmf_lv = run_analysis.NMF_comparison(str_name=[], percentile_threshold=90)

In [ ]:
plt.close("all")
fill_width=0.001
prominence_profile=0.0005

%matplotlib inline
fig_lv, ax_lv = plt.subplots(1, 1, figsize=(6, 4))
for l, line in enumerate(by_nmf_lv):
    line = line[run_analysis.range_ind[0]:run_analysis.range_ind[1]].copy()
    peaks = find_peaks(line, prominence=prominence_profile)[0]
    peaks = peaks * run_analysis.pixel_size_inv_Ang
    peaks = peaks + run_analysis.from_
    ax_lv.plot(run_analysis.x_axis, line, c=run_analysis.color_rep[l+1], label='lv %d'%(l+1))
    
    for ip, peak in enumerate(peaks):
        if peak >= run_analysis.from_ and peak <= run_analysis.to_:
            print(peak)
            ax_lv.axvline(peak, ls=':', lw=1.5, c='r')
            ax_lv.fill_between([peak-fill_width, peak+fill_width], 
                                  y1=np.max(line), 
                                  y2=np.min(line), 
                                  alpha=0.5, color='orange')
            ax_lv.text(peak, np.max(line), "%.3f"%(peak))
            
ax_lv.legend()
ax_lv.set_facecolor("lightgray")
fig_lv.tight_layout()
plt.show()

In [ ]:
# Save the mean profiles of the pixels with high coefficients loading vector by loading vector
# import hyperspy.api as hs
# by_nmf_lv = np.asarray(by_nmf_lv)
# print(by_nmf_lv.shape)
# by_nmf_lv = hs.signals.Signal1D(by_nmf_lv)
# by_nmf_lv.axes_manager[0].unit = "loading vector"
# by_nmf_lv.axes_manager[1].scale = run_analysis.pixel_size_inv_Ang
# by_nmf_lv.axes_manager[1].unit = "1/Å"
# by_nmf_lv.save('%s_mean_profiles_loading_vector.hspy'%run_analysis.formatted, overwrite=True)

In [ ]:
plt.close("all")
run_analysis.high_coeff_area_comparison()

In [ ]:
plt.close("all")
run_analysis.NMF_summary_save_specific(specific_data=[''], save=False, also_dp=True, # specific_data = ['a keyword in the file name']
                          log_scale_dp=True, also_tiff=False, 
                          fill_width=0.01, prominence_lv=0.001, 
                          prominence_profile=0.001)

In [ ]:
plt.close("all")
run_analysis.NMF_summary_save(save=False, also_tiff=False, also_dp=False, 
                              log_scale_dp=True, fill_width=0.005, 
                              prominence_lv=0.05, prominence_profile=0.005)

# Clustering of small areas

In [ ]:
plt.close("all")
# Detect small areas in each mask and calculate their centroid and boundary positions
# data_key='a keyword in the file name'
run_analysis.effective_small_area(data_key='110236', threshold_map="NMF", eps=4.0, min_sample=36)

In [ ]:
plt.close("all")
# Obtain the sum of 2D diffraction patterns for each small area
run_analysis.small_area_investigation(save=False, also_tiff=False, virtual_4D=True)

In [ ]:
plt.close("all")
# Check the overlap between small areas by loading vector (if 'threshold_map="NMF" above)
run_analysis.overlap_check()

In [ ]:
plt.close("all")
# Obtain single-phase regions for each data
run_analysis.sum_edx(edx_from=0.2, edx_to=18.0, offset=0.18, edx_scale=0.01, visual=False)
run_analysis.single_phase_investigation(visual=True, fig_save=False, 
                                        dp_shape=[515, 515], crop_ind=[0, 515, 0, 515],
                                        eps=4.5, min_sample=30)

In [ ]:
# LV area comparison
for i in range(len(run_analysis.subfolders)):
    for j, adr in enumerate(run_analysis.loaded_data_path[i]):
        data_key = os.path.dirname(run_analysis.loaded_data_path[i][j]).split("/")[-1]
        print(f'{run_analysis.subfolders[i]} | {data_key}')
        for lv in range(run_analysis.num_comp):
            area = (run_analysis.radial_var_split[i][j].axes_manager[0].scale**2 * 
                    run_analysis.num_lv_pixel_split[i][j]["nominal_LV%d"%(lv+1)])
            print(f'LV{lv+1} area = {area} {run_analysis.radial_var_split[i][j].axes_manager[0].units}^2')

In [ ]:
# Save the averaged diffraction patterns of single-phase areas
for lv in range(run_analysis.num_comp):
    hs_save = hs.signals.Signal2D(run_analysis.dp_storage['nominal_LV%d'%(lv+1)])
    print(hs_save)
    hs_save.axes_manager[0].unit = "diffraction pattern"
    hs_save.axes_manager[1].scale = run_analysis.pixel_size_inv_Ang
    hs_save.axes_manager[1].unit = "1/Å"
    hs_save.axes_manager[2].scale = run_analysis.pixel_size_inv_Ang
    hs_save.axes_manager[2].unit = "1/Å"
    hs_save.save("%s_diffraction_pattern_LV%d_clustering.hspy"%(run_analysis.formatted, lv+1), overwrite=True)

In [ ]:
# Save the mean profiles of the pixels with high coefficients loading vector by loading vector
by_nmf_lv = []
for lv in range(run_analysis.num_comp):
    by_nmf_lv.append(run_analysis.mean_rvp['nominal_LV%d'%(lv+1)]/run_analysis.num_pixel['nominal_LV%d'%(lv+1)])
by_nmf_lv = np.asarray(by_nmf_lv)
print(by_nmf_lv.shape)
# by_nmf_lv_save = hs.signals.Signal1D(by_nmf_lv)
# by_nmf_lv_save.axes_manager[0].unit = "loading vector"
# by_nmf_lv_save.axes_manager[1].scale = run_analysis.pixel_size_inv_Ang
# by_nmf_lv_save.axes_manager[1].unit = "1/Å"
# by_nmf_lv_save.save('%s_mean_profiles_LV_clustering.hspy'%run_analysis.formatted, overwrite=True)

In [ ]:
for i in range(len(run_analysis.subfolders)):
    fig, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=300)
    total_mean = np.mean(run_analysis.radial_var_sum_split[i], axis=0)
    ax.plot(run_analysis.x_axis, total_mean[run_analysis.range_ind[0]:run_analysis.range_ind[1]], 'k:')
    # ax.plot(run_analysis.x_axis, total_mean[run_analysis.range_ind[0]:run_analysis.range_ind[1]], 'k*')
    ax.set_facecolor("darkgray")
    ax_twin = ax.twinx()
    for lv, line in enumerate(by_nmf_lv):
        ax_twin.plot(run_analysis.x_axis, 
               line[run_analysis.range_ind[0]:run_analysis.range_ind[1]], c=run_analysis.color_rep[lv+1], label='lv %d'%(lv+1))
    
    fig.tight_layout()
    plt.show()

In [ ]:
# Save the mean edx spectra of the pixels with high coefficients loading vector by loading vector

run_analysis.sum_edx(edx_from=0.2, edx_to=18.0, offset=0.18, edx_scale=0.01, visual=False)

edx_by_nmf_lv = []
for lv in range(run_analysis.num_comp):
    edx_by_nmf_lv.append(run_analysis.mean_edx['nominal_LV%d'%(lv+1)]/run_analysis.num_pixel['nominal_LV%d'%(lv+1)])
edx_by_nmf_lv = np.asarray(edx_by_nmf_lv)
print(edx_by_nmf_lv.shape)
# edx_by_nmf_lv_save = hs.signals.Signal1D(edx_by_nmf_lv)
# edx_by_nmf_lv_save.set_signal_type("EDS_TEM")
# edx_by_nmf_lv_save.axes_manager[0].name = "loading vector"
# edx_by_nmf_lv_save.axes_manager[0].units = "LV"
# edx_by_nmf_lv_save.axes_manager[1].scale = run_analysis.edx_scale
# edx_by_nmf_lv_save.axes_manager[1].offset = -run_analysis.edx_offset
# edx_by_nmf_lv_save.axes_manager[1].units = "keV"
# edx_by_nmf_lv_save.axes_manager[1].name = "Energy"
# print(edx_by_nmf_lv_save)
# print(edx_by_nmf_lv_save.axes_manager)
# edx_by_nmf_lv_save.save('%s_mean_edx_LV_clustering.hspy'%run_analysis.formatted, overwrite=True)

In [ ]:
plt.close("all")

fig_lv, ax_lv = plt.subplots(1, 1, figsize=(12, 4), dpi=300)
for lv, line in enumerate(edx_by_nmf_lv):
    ax_lv.plot(run_analysis.edx_range[run_analysis.edx_range_ind[0]:run_analysis.edx_range_ind[1]], 
               line[run_analysis.edx_offset_ind[0]:run_analysis.edx_offset_ind[1]]+0.0001*lv, 
               c=run_analysis.color_rep[lv+1], label='lv %d'%(lv+1))
# ax_lv.legend()
ax_lv.tick_params(axis="both", labelsize=15)
ax_lv.set_facecolor("darkgray")
ax_lv.set_ylim(-0.00005, 0.0012)
# fig_lv.suptitle("Compare the mean of EDX spectra between loading vectors")
fig_lv.tight_layout()
plt.show()

# Concurrent EDX analysis

In [ ]:
plt.close("all")
run_analysis.edx_count()

In [ ]:
plt.close("all")
# energy scale: [keV]
# EDX intensity map and sum of EDX spectra
# The EDX calibration information is not parsed at the moment
# So the energy range must be specified using 'edx_from', 'edx_to' and 'edx_scale'
# The edx spectra can be shifted by changing 'offset'
run_analysis.sum_edx(edx_from=0.2, edx_to=18.0, offset=0.18, edx_scale=0.01, visual=True, total_edx=True)

In [ ]:
plt.close("all")
# sum of EDX spectra for the pixels with high NMF coefficients
run_analysis.sum_edx(edx_from=0.2, edx_to=18.0, offset=0.18, edx_scale=0.01, visual=False)
edx_by_nmf_lv = run_analysis.edx_classification(threshold_map='NMF')

In [ ]:
plt.close("all")
%matplotlib inline
run_analysis.sum_edx(edx_from=0.6, edx_to=18.0, offset=0.18, edx_scale=0.01, visual=False)
fig_lv, ax_lv = plt.subplots(1, 1, figsize=(12, 4))
for l, line in enumerate(edx_by_nmf_lv):
    ax_lv.plot(run_analysis.edx_range[run_analysis.edx_range_ind[0]:run_analysis.edx_range_ind[1]], 
               line[run_analysis.edx_offset_ind[0]:run_analysis.edx_offset_ind[1]]+0.00005*l, c=run_analysis.color_rep[l+1], label='lv %d'%(l+1))
ax_lv.legend()
ax_lv.set_facecolor("lightgray")
fig_lv.suptitle("Compare the mean of EDX spectra between loading vectors")
fig_lv.tight_layout()
plt.show()

In [ ]:
# Save the mean edx spectra of the pixels with high coefficients loading vector by loading vector
# import hyperspy.api as hs
# edx_by_nmf_lv = np.asarray(edx_by_nmf_lv)
# print(edx_by_nmf_lv.shape)
# edx_by_nmf_lv = hs.signals.Signal1D(edx_by_nmf_lv)
# edx_by_nmf_lv.axes_manager[0].unit = "loading vector"
# edx_by_nmf_lv.axes_manager[1].scale = run_analysis.edx_scale
# edx_by_nmf_lv.axes_manager[1].offset = run_analysis.edx_offset
# edx_by_nmf_lv.axes_manager[1].unit = "keV"
# edx_by_nmf_lv.save('%s_mean_edx_loading_vector.hspy'%run_analysis.formatted, overwrite=True)

# High variances

In [ ]:
plt.close("all")
# Peak detection
# Please refer to SciPy 'find_peaks' for details
# scattering vector range -> [peak_position-half_width, peak_position+half_width]
half_width = 0.005
run_analysis.scattering_range_of_interest(fill_width=half_width,
                                         profile_type="variance",
                                         prominence=0.01,
                                         height=None,
                                         width=None,
                                         distance=None,
                                         threshold=None)

In [ ]:
plt.close("all")
# Variance maps for the specified scattering vector range
# Average and standard deviation of variances for the specified scattering vector range
# Threshold maps - the pixels with high variances will be 1, otherwise 0
### abs_threshold - > absolute threshold value
### percentile_threshold -> if 90, only the pixels with the 10% highest variances remain

# This will show the results for each peak detected in the cell above
# The loop is not necessary
sum_radial_list = []
for i, peak in enumerate(run_analysis.peak_sub[run_analysis.subfolders[0]]):
    peak_selected = peak
    print(run_analysis.subfolders[0], ", peak No. %d - range: %.3f ~ %.3f"%(i+1, peak-half_width, peak+half_width))
    run_analysis.variance_map(sv_range=[peak_selected-half_width, peak_selected+half_width])
    tmp_sum_radial = run_analysis.high_variance_map(percentile_threshold=90)
    print("threshold value to determine the high variances: %.3f"%run_analysis.abs_threshold)
    sum_radial_list.append(tmp_sum_radial)

    # optional - if you want to see the mean 2D diffraction patterns for individual data
    # It will take a long time due to loading each 4D data
    # run_analysis.summary_save(save=False, 
    #                           also_dp=True,
    #                           log_scale_dp=True)

In [ ]:
# Save the mean profiles of the pixels with high variances peak by peak
# import hyperspy.api as hs
# sum_radial_list = np.asarray(sum_radial_list)
# print(sum_radial_list.shape)
# sum_radial_list = hs.signals.Signal1D(sum_radial_list)
# sum_radial_list.axes_manager[0].unit = "peak"
# sum_radial_list.axes_manager[1].scale = run_analysis.pixel_size_inv_Ang
# sum_radial_list.axes_manager[1].unit = "1/Å"
# sum_radial_list.save('%s_mean_profiles_by_peak.hspy'%run_analysis.formatted, overwrite=True)

In [ ]:
plt.close("all")
# Compare the mean of radial profiles
# run_analysis.basic_setup(str_path, 0.1, 1.0, broadening=0.01) # specify the scattering vector range
fig, ax = plt.subplots(1, 1, figsize=(12, 4))

# sel_peak_num = np.array([1, 2, 3]) - 1 # select a few peaks of interest
sel_peak_num = np.arange(len(run_analysis.peak_sub[run_analysis.subfolders[0]])) # for all the peaks

for i in sel_peak_num:
    ax.plot(run_analysis.x_axis, 
               sum_radial_list[i][int(run_analysis.from_/run_analysis.pixel_size_inv_Ang):int(run_analysis.to_/run_analysis.pixel_size_inv_Ang)], 
               c=run_analysis.color_rep[i], label="peak No.%d"%(i+1))
ax.legend()
ax.set_xlabel("Scattering vector (1/Å)")
ax.set_facecolor("lightgray")
fig.tight_layout()
plt.show()